### Setup and load torch

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
import sys,cv2
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from Utils.utils import *
from Utils.center_images import *
from Utils.f1_score import *
from ipywidgets import interact
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

import torch
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

import deepdish as dd

import timeit,gc

data_folder = 'D:/data/HPA/all/'
model_folder = 'D:/data/HPA/models/'
LOSS = 'F1'
USE_SMALL_IMAGES = True
USE_ALL_CHANNELS = True
np.random.seed(42)

print("Using GPU:",torch.cuda.is_available())
print("Using device ",torch.cuda.get_device_name(0))
print("Done.")

### Load training data

In [ ]:
%%time
filename = "_augmented_"
if USE_SMALL_IMAGES:
    filename = filename + "small.h5"
else:
    filename = filename + ".h5"
if USE_ALL_CHANNELS:
    filename = "all_channel" + filename
else:
    filename = "poi" + filename

d = dd.io.load(data_folder+filename)
    
X = d['X'].astype(np.float32) / 255.0 # torch likes float images

y = d['labels']

idx = np.arange(y.shape[0])
np.random.shuffle(idx)
X = X[idx]
y = y[idx]
 
old = X[1].squeeze()
X = center_images(X)
    
print("Shapes are:")
print(X.shape, "MEAN=",np.mean(X), "STD=",np.std(X))
print(y.shape)

print("Done")

### Initialize (and load) model

In [ ]:
LOAD_OLD_MODEL = False

#Setup resnet model
net = models.squeezenet1_1(pretrained=True)

#freeze feature layers
for layer in net.features:
    layer.requires_grad = False

old_weights = net.features[0].weight #remember weights even though we reinit first layer

if USE_ALL_CHANNELS:     
    net.features[0] = nn.Conv2d(4, 64, kernel_size=3, stride=2) # adapt to single channel input
    net.features[0].weights = torch.stack([old_weights[:,0,:,:], old_weights[:,0,:,:], old_weights[:,0,:,:], old_weights[:,0,:,:]]).permute([1,0,2,3])  
    net.requires_grad = False #freeze first layer
else:
    net.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=2) # adapt to single channel input
    net.features[0].weights = torch.stack([old_weights[:,0,:,:]]).permute([1,0,2,3])
    net.requires_grad = False #freeze first layer

net.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, 28, kernel_size=1), #we are missing a relu here, because relu kills the symmetry we want if we use a sigmoid later
    nn.AvgPool2d(13)
)

net.forward = lambda x: net.classifier(net.features(x)).view(x.size(0), 28)

if LOAD_OLD_MODEL:
    if USE_SMALL_IMAGES:
        net.load_state_dict(torch.load(model_folder+"squeeze_small.model"))
    else:
        net.load_state_dict(torch.load(model_folder+"squeeze.model")) 
print("Done.")

### Initialize loss and optimizer

In [ ]:
from Utils.f1_loss import *
from Utils.f1_loss import *

if LOSS == 'F1':
    #Setup custom F1 Loss
    criterion = F1_Loss()
else:
    #Setup BCE Loss and optimizer
    weights = np.asarray(y.shape[0]/np.sum(y,axis=0))
    for idx,weight in enumerate(weights):
        if isinf(weight):
            weights[idx] = y.shape[0]
    print("Weights = ",weights) # we weight classes given their skewed distribution
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(weights,dtype=torch.float))

optimizer = optim.Adam(net.parameters(),lr=0.000075)

print("Done.")

### Run training

In [ ]:
gc.collect() 

epochs = 1
batch_size = 100
time_per_epoch = 0
net.train()

iterations_per_epoch = np.ceil(X.shape[0] / batch_size).astype(int)
runtime = 0

for epoch in range(epochs):
    running_loss,running_f1,average_targets_predicted = 0,0,0
      
    #reshuffle
    idx = np.arange(y.shape[0])
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
        
    for i in range(iterations_per_epoch):
        real_batch_size = np.minimum(batch_size,X.shape[0] - i*batch_size)
        if real_batch_size == 0: #in case X.shape[0] is divisible by batch size this is required
            real_batch_size = batch_size

        start = timeit.default_timer() #measure time
        
        start_idx = i * batch_size 
        X_batch = torch.tensor(X[start_idx:start_idx+real_batch_size].transpose(0,3,1,2))
        y_batch = torch.tensor(y[start_idx:start_idx+real_batch_size].astype(np.float32),dtype=torch.float)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(X_batch)
        
        if LOSS == 'F1':
            outputs = torch.sigmoid(outputs)
        
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        #compute F1 scores
        label = y_batch.detach().numpy().astype(np.bool)
        logits = outputs.detach().numpy() > 0.5
        average_targets_predicted = 0.5 * average_targets_predicted + 0.5 * np.sum(logits) / batch_size
        running_f1 += f1_score(label,logits)
        
        #measure runtime
        stop = timeit.default_timer()
        time_per_epoch = 0.5 * time_per_epoch + 0.5 * (stop-start) * iterations_per_epoch
        runtime += (stop-start)
        
        #print performance metrics
        if i % 5 == 0:
            print('[epoch = (%d/%d), iteration = (%3d/%d), time = %3ds, est. time per epoch = %5ds] \t loss = %.5f ## F1 = %.5f pred/img = %.3f'\
                  %(epoch + 1, epochs,i + 1, iterations_per_epoch, runtime, time_per_epoch, running_loss / (i+1), running_f1 / (i+1), average_targets_predicted))


### Load validation data

In [ ]:
%%time
filename = "_6_"
if USE_SMALL_IMAGES:
    filename = filename + "small.h5"
else:
    filename = filename + ".h5"
if USE_ALL_CHANNELS:
    filename = "all_channel" + filename
else:
    filename = "poi" + filename

d = dd.io.load(data_folder+filename)
    
Xval = center_images(d['X'].astype(np.float32) / 255.0) # torch likes float images
yval = d['labels']

idx = np.arange(yval.shape[0])

np.random.shuffle(idx)
Xval = Xval[idx]
yval = yval[idx]
print("Done")

### Run validation

In [ ]:
# Run validation
gc.collect() 
net.eval()

batch_size = 67
iterations_per_epoch = np.ceil(Xval.shape[0] / batch_size).astype(int)
time_per_epoch,runtime,running_loss,running_f1 = 0,0,0,0

for i in range(iterations_per_epoch):
    start = timeit.default_timer() #measure time

    start_idx = i * batch_size 
    X_batch = torch.tensor(Xval[start_idx:start_idx+batch_size].transpose(0,3,1,2))
    y_batch = torch.tensor(yval[start_idx:start_idx+batch_size].astype(np.float32),dtype=torch.float)

    # forward + backward + optimize
    outputs = net(X_batch)

    loss = criterion(outputs, y_batch)
    
    running_loss += loss.item()

    #compute F1 scores
    act = torch.sigmoid(outputs)

    label = y_batch.detach().numpy().astype(np.bool)
    logits = act.detach().numpy() > 0.5
    print("Targets in batch = ",np.sum(label),"Predicted targets = ",np.sum(logits))
    running_f1 += f1_score(label,logits)

    #measure runtime
    stop = timeit.default_timer()
    time_per_epoch = 0.5 * time_per_epoch + 0.5 * (stop-start) * iterations_per_epoch
    runtime += (stop-start)
    #print performance metrics
    N = ((i+1)*batch_size)
    print('[iteration = (%3d/%d), time = %3ds, est. time per epoch = %5ds] \t loss = %.5f ## F1 = %.5f'\
          %(i + 1, iterations_per_epoch, runtime, time_per_epoch, running_loss / N, running_f1 / (i+1)))

### Save the trained model

In [ ]:
# Save model
if USE_SMALL_IMAGES:
    torch.save(net.state_dict(), model_folder+"squeeze_small.model")
else:
    torch.save(net.state_dict(),  model_folder+"squeeze.model")
print("Done.")

### Shutdown system (can be run after training and saving)

In [ ]:
import subprocess
cmdCommand = "shutdown -s"
process = subprocess.Popen(cmdCommand.split(), stdout=subprocess.PIPE)